## Analyzing borrowers’ risk of defaulting



In [1]:
import pandas as pd

In [2]:
bank= pd.read_csv("./datasets/credit_scoring_eng.csv")

In [3]:
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,masters degree,0,married,0,F,employee,0,253875.639453,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,112080.014102,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,145885.952297,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,267628.550329,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,158616.077870,to have a wedding


In [4]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

##### There seems to be missing values in both days_employed and total_income field which may be because the customer opted out from providing the employment information

###  Data preprocessing

Changing the negative values in children, days_employed and total_income column.

In [5]:
#funstion to return absolute value 
def change_to_positive(value):
    return abs(value)
    

In [6]:
bank['days_employed']=bank['days_employed'].apply(change_to_positive)
bank['total_income']=bank['total_income'].apply(change_to_positive)
bank['children']=bank['children'].apply(change_to_positive)

In [7]:
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,masters degree,0,married,0,F,employee,0,253875.639453,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,112080.014102,car purchase
2,0,5623.422610,33,Secondary Education,1,married,0,M,employee,0,145885.952297,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,267628.550329,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,158616.077870,to have a wedding


### Processing duplicates

In [8]:
bank['education'].unique()

array(['masters degree', 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', 'MASTERS DEGREE', 'bachelor degree',
       'primary education', 'Masters Degree', 'BACHELOR DEGREE',
       'Bachelor Degree', 'PRIMARY EDUCATION', 'Primary Education',
       'Academic Degree', 'ACADEMIC DEGREE', 'academic degree'],
      dtype=object)

##### The education field seems to have duplicates which may be because while filling the loan form different customers used different case and this can simply be solved by changing all the values in upper or lower case

In [9]:
#to change the education field to lower
def to_lower_value(value):
    return value.lower()

In [10]:
bank['education']=bank['education'].apply(to_lower_value)

In [11]:
bank['education'].unique()

array(['masters degree', 'secondary education', 'bachelor degree',
       'primary education', 'academic degree'], dtype=object)

In [12]:
bank['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [13]:
bank['income_type'].unique()

array(['employee', 'retiree', 'partner', 'civil servant', 'unempoyed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

### Conclusion

##### All the fields except the 'purpose' field is free from duplicates

### Lemmatization

In [14]:

bank['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car',
       'transactions with the residential real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get asupplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'property renovat

##### There are lot of duplicates in this field such as 'car purchase' and 'buying my own car' has same meaning and same general purpose but different customer choose different wording.
##### This can be fixed by using lemmatization method.

In [15]:
#importing libraries required for lemmatization
import nltk
from nltk.stem import WordNetLemmatizer


In [16]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\my
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\my
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
wordnet_lemma=WordNetLemmatizer()


In [18]:
#list of duplicate words in purpose field
word_list=['house','car','education','wedding','estate','property', 'university', 'educated','housing']

#To add a field to reduce the duplicates on 'purpose' field by categorizing togther similar values.    
for i in bank.index:
    words=nltk.word_tokenize(bank['purpose'].loc[i])
    lemmas=[wordnet_lemma.lemmatize(w,pos='n') for w in words]
    
    for word in word_list:
        check_word = wordnet_lemma.lemmatize(word,pos='n')
        if check_word in lemmas:
            #since university, educated and education are simiar words
            if (word == 'university') or (word == 'educated') :
                bank.loc[i,'purpose_short']='education'
            #since housing and house are simiar word
            elif (word == 'housing'):
                bank.loc[i,'purpose_short']='house'
            else:                
                bank.loc[i,'purpose_short']=word
    

In [19]:
null_fields=bank[bank['purpose_short'].isnull()]

In [20]:
null_fields

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_short


In [21]:
bank['purpose_short'].unique()

array(['house', 'car', 'education', 'wedding', 'estate', 'property'],
      dtype=object)

### Conclusion

##### After lemmatization we were able to categorize on the base of their general purpose .

### Processing missing values

In [22]:
null_fields=bank[bank.isnull().any(axis=1)]


In [23]:
null_fields.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_short
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,car
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,wedding


In [24]:
null_fields.shape

(2174, 13)

In [25]:
null_fields.groupby('income_type')['income_type'].agg('count')

income_type
civil servant     147
employee         1105
entrepreneur        1
partner           508
retiree           413
Name: income_type, dtype: int64

##### There are altogether 2174 rows with missing values and observing the data, the missing values seems to be random, it may be because of the problem with the system or may be because the customer opted out from providing the employment information.
##### Since 10% of the total data values is missing, droping the row is not an option.
##### The data is Quantitative so the best option is to fill the missing value with representative values(median) of the similar data. 

In [26]:
def find_median(education,income):
    result=[]
    #filtering those data that has similar education and income source
    to_find_median_data=bank[(bank['education']==education) & (bank['income_type']==income)].copy()
    result.append(to_find_median_data['days_employed'].median())
    result.append(to_find_median_data['total_income'].median())
    return result
    

In [27]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
purpose_short       21525 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


In [28]:
for i in bank[bank.isnull().any(axis=1)].index:    
    median_result=find_median(bank.iloc[i]['education'],bank.iloc[i]['income_type'])
    bank.loc[i,'days_employed']=median_result[0]
    bank.loc[i,'total_income']=median_result[1]
    

In [29]:
bank[bank.isnull().any(axis=1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_short


### Conclusion

##### The data no longer contains any missing field

### Data type replacement

Changing the days_employed and total_income field from float type to integer type.

In [30]:
def data_type_int(value):
    return int(value)

In [31]:
bank['days_employed']=bank['days_employed'].apply(data_type_int)
bank['total_income']=bank['total_income'].apply(data_type_int)

In [32]:
bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_short
0,1,8437,42,masters degree,0,married,0,F,employee,0,253875,purchase of the house,house
1,1,4024,36,secondary education,1,married,0,F,employee,0,112080,car purchase,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,145885,purchase of the house,house
3,3,4124,32,secondary education,1,married,0,M,employee,0,267628,supplementary education,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,158616,to have a wedding,wedding
5,0,926,27,masters degree,0,civil partnership,1,M,partner,0,255763,purchase of the house,house
6,0,2879,43,masters degree,0,married,0,F,partner,0,240525,housing transactions,house
7,0,152,50,secondary education,1,married,0,M,employee,0,135823,education,education
8,2,6929,35,masters degree,0,civil partnership,1,F,employee,0,95856,having a wedding,wedding
9,0,2188,41,secondary education,1,married,0,M,employee,0,144425,purchase of the house for my family,house


### Conclusion

##### The data type for the 'days_employed' field and 'total_income' field is changed to make the calculation in further analysis easier by type casting.

### Looking for values that is unlikely to be true

In [33]:
bank['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

In [34]:
bank[bank['children']==20].count()

children            76
days_employed       76
dob_years           76
education           76
education_id        76
family_status       76
family_status_id    76
gender              76
income_type         76
debt                76
total_income        76
purpose             76
purpose_short       76
dtype: int64

In [35]:
#droping the rows where the customer has 20 children as there are only 76 such field which is about 3-4% of our data
bank=bank[bank['children']!=20]

In [36]:
bank['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [37]:
bank[bank['dob_years']==0].count()

children            100
days_employed       100
dob_years           100
education           100
education_id        100
family_status       100
family_status_id    100
gender              100
income_type         100
debt                100
total_income        100
purpose             100
purpose_short       100
dtype: int64

In [38]:
#droping the rows where the customer is 0 years old as there are only 100 such field which is about 4-5% of our data
bank= bank[bank['dob_years']!=0]

###  Answer these questions

### - Is there a relation between having kids and repaying a loan on time?

In [39]:
result=bank.groupby(['children','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)

children  debt
0         0       13022
          1        1058
1         0        4407
          1         442
2         0        1848
          1         194
3         0         301
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64 


Percentage 


children  debt
0         0        92.485795
          1         7.514205
1         0        90.884718
          1         9.115282
2         0        90.499510
          1         9.500490
3         0        91.768293
          1         8.231707
4         0        90.243902
          1         9.756098
5         0       100.000000
Name: debt, dtype: float64


In [40]:
def have_children(value):
    if value == 0:
        return 'No'
    else:
        return 'Yes'

In [41]:
bank['have_children']=bank['children'].apply(have_children)
result=bank.groupby(['have_children','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)



have_children  debt
No             0       13022
               1        1058
Yes            0        6602
               1         667
Name: debt, dtype: int64 


Percentage 


have_children  debt
No             0       92.485795
               1        7.514205
Yes            0       90.824047
               1        9.175953
Name: debt, dtype: float64


### Conclusion

##### Since, having children and not having children has almost same rate of loan default, there is no relation between having kids and repaying loan.

### - Is there a relation between marital status and repaying a loan on time?

In [42]:
result=bank.groupby(['family_status','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)

family_status      debt
civil partnership  0        3761
                   1         383
divorced           0        1099
                   1          84
married            0       11359
                   1         924
unmarried          0        2516
                   1         272
widow / widower    0         889
                   1          62
Name: debt, dtype: int64 


Percentage 


family_status      debt
civil partnership  0       90.757722
                   1        9.242278
divorced           0       92.899408
                   1        7.100592
married            0       92.477408
                   1        7.522592
unmarried          0       90.243902
                   1        9.756098
widow / widower    0       93.480547
                   1        6.519453
Name: debt, dtype: float64


### Conclusion

##### Since, all marital status has same rate of loan default, there is no signinficant relation between marital status and repaying loan.

### - Is there a relation between income level and repaying a loan on time?

In [43]:
bank['total_income'].describe()

count    2.134900e+04
mean     1.653901e+05
std      9.832614e+04
min      2.066700e+04
25%      1.077190e+05
50%      1.434930e+05
75%      1.980680e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [44]:
#Function to categorize the data based on income
def income_level(value):
    if value < 50000:
        return 'very low'
    elif value>=50000 and value <120000:
        return 'low'
    elif value>=120000 and value <190000:
        return 'average'
    else:
        return 'high'

In [45]:
bank['income_level']=bank['total_income'].apply(income_level)

In [46]:
result=bank.groupby(['income_level','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)

income_level  debt
average       0       7602
              1        725
high          0       5463
              1        423
low           0       6212
              1        554
very low      0        347
              1         23
Name: debt, dtype: int64 


Percentage 


income_level  debt
average       0       91.293383
              1        8.706617
high          0       92.813456
              1        7.186544
low           0       91.812001
              1        8.187999
very low      0       93.783784
              1        6.216216
Name: debt, dtype: float64


### Conclusion

##### Since, all the income level seems to have almost similar rate of loan repayment, income level does not seem to have relation with the ontime loan repayment.

### - How do different loan purposes affect on-time repayment of the loan?

In [47]:
result=bank.groupby(['purpose_short','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)

purpose_short  debt
car            0       3880
               1        398
education      0       3620
               1        369
estate         0       4112
               1        335
house          0       2957
               1        219
property       0       2911
               1        223
wedding        0       2144
               1        181
Name: debt, dtype: int64 


Percentage 


purpose_short  debt
car            0       90.696587
               1        9.303413
education      0       90.749561
               1        9.250439
estate         0       92.466832
               1        7.533168
house          0       93.104534
               1        6.895466
property       0       92.884493
               1        7.115507
wedding        0       92.215054
               1        7.784946
Name: debt, dtype: float64


### Conclusion

##### Almost all the purpose has similar rate of repayment and hence the loan repayment does not seem to have relation with the purpose for which loan was taken.  However, those who takes loan to buy house and property seems to pay loan on time in compare to others, may be its because the loan amount is quite high in those cases.

#### - How do different loan purposes affect on-time repayment of the loan?

In [48]:
bank['dob_years'].describe()

count    21349.000000
mean        43.501429
std         12.250774
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [49]:
def age_bin(value):
    if value <=33:
        return 'Young'
    elif value > 33 and value <= 43:
        return 'Adult'
    elif value > 43 and value <= 53:
        return 'Senior'
    else:
        return 'Old'

In [50]:
bank['age_group']=bank['dob_years'].apply(age_bin)

In [51]:
result=bank.groupby(['age_group','debt'])['debt'].agg('count')
print(result,'\n\n')

print("Percentage",'\n\n')
result_percentage = result.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))
print(result_percentage)

age_group  debt
Adult      0       5291
           1        494
Old        0       4983
           1        293
Senior     0       4580
           1        352
Young      0       4770
           1        586
Name: debt, dtype: int64 


Percentage 


age_group  debt
Adult      0       91.460674
           1        8.539326
Old        0       94.446550
           1        5.553450
Senior     0       92.862936
           1        7.137064
Young      0       89.058999
           1       10.941001
Name: debt, dtype: float64


### Conclusion

##### The age seems to be a factor which affects the loan repayment time. Older people tends to pay the loan on time than younger.

### General conclusion

In [52]:
bank.groupby('debt')['debt'].agg('count')


debt
0    19624
1     1725
Name: debt, dtype: int64

##### In general around 91% of customers pays loan on time. Also, the older the customer the more likely he/she will pay the loan on time and if the loan amount is higher then the customer is more likely to pay the loan amount on time.